In [5]:
import os
import numpy as np
import cv2
import fnmatch
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from pathlib  import Path
import h5py
from glob import glob

input_preds ='/home/ext_jang_inyeop_mayo_edu/hwangt-share/datasets/Pathology_Slides/CLASSIC_Stomach_Cancer_Image/immune_subtype'
MSI_PATH =f'{input_preds}/MSI_prediction/pred_heatmaps'
os.makedirs(os.path.join(input_preds, 'overlay'), exist_ok=True)

In [6]:
files  = glob(f"{input_preds}"+'/tils/*.png') #contour h5y files by create_patches_fn.py from pickles by /home/jupyter/src/Mask_annotation_jang.ipynb
#os.mkdirs(f'{input_preds}/overlay',ext_ok=True)
with open(f'{input_preds}/result_Tumor_tils_density_test.csv', 'w') as outfile :
    outfile.write('file_name,immune_infl,immune_excl,immune_des\n')
    for filename in files :
        filename =filename.split('/')[-1]
        slide_name = filename.split('.')[0] #.replace('.svs', '_gray.png')

        stroma = input_preds + '/stroma/' + filename
        til = input_preds + '/tils/' + filename            
        tumor = input_preds + '/tumor/' + filename

        if not (os.path.exists(tumor) and os.path.exists(til) and os.path.exists(stroma)) : continue

        tumor_img_gray = cv2.imread(tumor, cv2.IMREAD_GRAYSCALE)
        til_img_gray = cv2.imread(til, cv2.IMREAD_GRAYSCALE)
        stroma_img_gray = cv2.imread(stroma, cv2.IMREAD_GRAYSCALE)

        til_img_gray[til_img_gray < 128] = 0
        til_img_gray[til_img_gray >= 128] = 1

        stroma_img_gray[stroma_img_gray < 128] = 0
        stroma_img_gray[stroma_img_gray >= 128] = 1

        tumor_img_gray = cv2.resize(tumor_img_gray, (til_img_gray.shape[1],til_img_gray.shape[0]))
        tumor_img_gray[tumor_img_gray < 128] = 0
        tumor_img_gray[tumor_img_gray >= 128] = 1

        out = np.zeros((tumor_img_gray.shape[0], tumor_img_gray.shape[1], 3))

        out[:,:,0] = tumor_img_gray #R
        out[:,:,1] = stroma_img_gray #G
        out[:,:,2] = til_img_gray #B

        cv2.imwrite(f'{input_preds}/overlay/{slide_name}__tumor.png', tumor_img_gray*255)
        cv2.imwrite(f'{input_preds}/overlay/{slide_name}__stroma.png', stroma_img_gray*255)
        cv2.imwrite(f'{input_preds}/overlay/{slide_name}__til.png', til_img_gray*255)
        cv2.imwrite(f'{input_preds}/overlay/{slide_name}__comb.png', out*255)

        # msi_pred_path =f'{input_preds}/MSI_prediction/pred_heatmap_Stomach_Immunotherapy_stmary/{slide_name}_pred_heatmap.png'
        # if os.path.exists(msi_pred_path):
        #     msi_pred=cv2.imread( msi_pred_path, cv2.IMREAD_COLOR)
        #     msi_pred =cv2.resize(msi_pred, (out.shape[1], out.shape[0]))
        #     blend = cv2.addWeighted((out*255).astype(np.float32),0.5,msi_pred.astype(np.float32),0.5,0.0)
        #     cv2.imwrite(f'{input_preds}/overlay/{slide_name}__comb_msi.png', blend)

        tissue_mask_img_gray = cv2.bitwise_or(til_img_gray, tumor_img_gray)
        tissue_mask_img_gray = cv2.bitwise_or(tissue_mask_img_gray, stroma_img_gray)
        cv2.imwrite(f'{input_preds}/overlay/{slide_name}__mask.png', tissue_mask_img_gray*255)

        overlap_tt = til_img_gray + tumor_img_gray # overlap TIL and Tumor
        overlap_ts=til_img_gray + stroma_img_gray # overlap TIL and stroma
        overlap_tst=tissue_mask_img_gray + til_img_gray # overlap Tissue and TIL

        # binarize overlapped images
        c_overlap_tt = overlap_tt > 1 
        c_overlap_ts=overlap_ts > 1
        c_overlap_tst=overlap_tst > 1

        imm_inf=0
        imm_excl=0
        imm_des=0
        c_tissue= 0

        i=0

        M,N = 17,17
        
        tiles = [tissue_mask_img_gray[x:x+M,y:y+N] for x in range(0,tissue_mask_img_gray.shape[0],M) for y in range(0,tissue_mask_img_gray.shape[1],N)]
        rows, cols = tissue_mask_img_gray.shape
        img_color = np.dstack((til_img_gray, stroma_img_gray, tumor_img_gray)) * 255
        comb_img = img_color.copy()

        msi_pred_path =f'{MSI_PATH}/{slide_name}_pred_heatmap.png'
        msi_flag = False
        if os.path.exists(msi_pred_path):
            msi_pred=cv2.imread( msi_pred_path, cv2.IMREAD_COLOR)
            msi_pred =cv2.resize(msi_pred, (comb_img.shape[1], comb_img.shape[0]))
            comb_msi = cv2.addWeighted(comb_img.astype(np.float32),0.4,msi_pred.astype(np.float32),0.6,0.0)
            msi_flag = True
             
            cv2.imwrite(f'{input_preds}/overlay/{slide_name}__comb_msi.png', cv2.cvtColor(comb_msi, cv2.COLOR_RGB2BGR))

        for x in range(0,tissue_mask_img_gray.shape[0],M):
            for y in range(0,tissue_mask_img_gray.shape[1],N):
                #print(i)
                #print(tiles[i].sum())
                if (tiles[i].sum())>0:
                    tissue_sum=tiles[i].sum()
                    c_tissue+=1 #count the number of tissue tiles
                    overlap_count_tt = sum(map(lambda x : x == True, c_overlap_tt[x:x+M,y:y+N]))#count the overlap ratio
                    overlap_count_ts = sum(map(lambda x : x == True, c_overlap_ts[x:x+M,y:y+N]))
                    overlap_count_tst = sum(map(lambda x : x == True, c_overlap_tst[x:x+M,y:y+N]))
                    sum_overlap_tt = np.sum(overlap_count_tt)
                    sum_overlap_ts = np.sum(overlap_count_ts)
                    sum_overlap_tst = np.sum(overlap_count_tst)
                    #print(sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]),sum_overlap_ts/np.count_nonzero(stroma_img[x:x+M,y:y+N]),sum_overlap_tst)

                    if np.isnan(sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]))==False and sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]) > 0.2:
                        imm_inf+=1                       
                        # start_point = (y, x - M)  topleft
                        # end_point = (y + N, x) +  bottomright
                        # color = (255, 0, 255) magenta
                        # thickness = 1
                        img_color = cv2.rectangle(img_color, (y, x + M), (y + N, x), (255, 0, 255), 2)
                        if msi_flag: comb_msi = cv2.rectangle(comb_msi, (y, x + M), (y + N, x), (255, 0, 255), 2)

                        
                       
                    elif np.isnan(sum_overlap_ts/np.count_nonzero(stroma_img_gray[x:x+M,y:y+N]))==False and sum_overlap_ts/np.count_nonzero(stroma_img_gray[x:x+M,y:y+N]) >0.1:
                        imm_excl+=1
                        # color = (255, 165, 0) orange
                        img_color = cv2.rectangle(img_color, (y, x + M), (y + N, x), (255, 165, 0), 2)
                        if msi_flag: comb_msi = cv2.rectangle(comb_msi, (y, x + M), (y + N, x), (255, 165, 0), 2)

                    
                    else:
                        imm_des+=1
                        # color = (0, 100, 1000) cyan
                        img_color = cv2.rectangle(img_color, (y, x + M), (y + N, x), (0, 255, 255), 2)
                        if msi_flag: comb_msi = cv2.rectangle(comb_msi, (y, x + M), (y + N, x), (0, 255, 255), 2)


                i+=1
            
        immune_subtype =  cv2.cvtColor(img_color, cv2.COLOR_RGB2BGR)
        cv2.imwrite(f'{input_preds}/overlay/{slide_name}__immune_sub.png',immune_subtype)
        if msi_flag: cv2.imwrite(f'{input_preds}/overlay/{slide_name}__immune_msi.png', cv2.cvtColor(comb_msi, cv2.COLOR_RGB2BGR))

                        
        if c_tissue == 0 : continue
        outfile.write(filename
                      + ',' + str(imm_inf/c_tissue)
                      + ',' + str(imm_excl/c_tissue)
                      + ',' + str(imm_des/c_tissue) + '\n')
    outfile.close()

/tmp/ipykernel_86397/679146825.py:99: RuntimeWarning: invalid value encountered in long_scalars
  if np.isnan(sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]))==False and sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]) > 0.2:
/tmp/ipykernel_86397/679146825.py:110: RuntimeWarning: invalid value encountered in long_scalars
  elif np.isnan(sum_overlap_ts/np.count_nonzero(stroma_img_gray[x:x+M,y:y+N]))==False and sum_overlap_ts/np.count_nonzero(stroma_img_gray[x:x+M,y:y+N]) >0.1:
